In [10]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Cài đặt đúng phiên bản TensorFlow và numpy
# !pip install -q numpy==1.23.5
# !pip install -q tensorflow==2.12.0 keras==2.12.0

# # Clone lại keras-bert bản tương thích
# !git clone https://github.com/CyberZHG/keras-bert.git
# %cd keras-bert
# !pip install .
# %cd /content



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
path = "/content/drive/My Drive/SentimentFoody/"

In [12]:
import pickle
import pandas as pd
df = pd.read_csv(path+"NTC_SV/NTC_SV_train.csv")
df_test = pd.read_csv(path+'NTC_SV/NTC_SV_test.csv')
df_test = df_test.dropna()
df = df.dropna()

In [13]:
data_train = df.review.to_list()
label_train = df.label.to_list()
data_test = df_test.review.tolist()
label_test = df_test.label.tolist()


In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf

import codecs

from keras_bert import Tokenizer, load_trained_model_from_checkpoint, load_vocabulary
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Input, SpatialDropout1D, Bidirectional, Concatenate
from tensorflow.keras.layers import Conv1D,GlobalMaxPooling1D, GlobalAveragePooling1D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow.keras as keras
from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K

import os


In [ ]:
# !wget -q https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
# !unzip -o multi_cased_L-12_H-768_A-12.zip

Archive:  multi_cased_L-12_H-768_A-12.zip
   creating: multi_cased_L-12_H-768_A-12/
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: multi_cased_L-12_H-768_A-12/bert_config.json  


In [ ]:
# !zip -r multi_cased_L-12_H-768_A-12.zip multi_cased_L-12_H-768_A-12


updating: multi_cased_L-12_H-768_A-12/ (stored 0%)
updating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta (deflated 92%)
updating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001 (deflated 7%)
updating: multi_cased_L-12_H-768_A-12/vocab.txt (deflated 45%)
updating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index (deflated 68%)
updating: multi_cased_L-12_H-768_A-12/bert_config.json (deflated 55%)


In [ ]:
# from google.colab import files
# files.download("multi_cased_L-12_H-768_A-12.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
pretrained_path = 'multi_cased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [16]:
SEQ_LEN = 256

In [17]:
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    seq_len=SEQ_LEN,
    output_layer_num=4,
    trainable=True
)

/usr/local/lib/python3.11/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [18]:
LR=2e-5

In [19]:
class NonMasking(keras.layers.Layer):
    def __init__(self, **kwargs):
        self.supports_masking = True
        super(NonMasking, self).__init__(**kwargs)

    def build(self, input_shape):
        input_shape = input_shape

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        return x

    def get_output_shape_for(self, input_shape):
        return input_shape

In [20]:
# Nếu chưa khai báo thì làm lại cho chắc
inputs = model.inputs
bert_out = model.outputs[0]  # Đảm bảo là Tensor 3D: (None, 256, 3072)

# Giữ nguyên mask
class NonMasking(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(NonMasking, self).__init__(**kwargs)
        self.supports_masking = True

    def call(self, inputs, mask=None):
        return inputs

    def compute_mask(self, inputs, mask=None):
        return None

bert_out = NonMasking()(bert_out)

# Dropout
text_embed = SpatialDropout1D(0.2)(bert_out)

# Shift trái và phải
def shift_left(x):
    zeros = K.zeros_like(x[:, :1, :])  # shape (batch, 1, dim)
    return K.concatenate([zeros, x[:, :-1, :]], axis=1)

def shift_right(x):
    zeros = K.zeros_like(x[:, :1, :])
    return K.concatenate([x[:, 1:, :], zeros], axis=1)

l_embedding = Lambda(shift_left)(text_embed)
r_embedding = Lambda(shift_right)(text_embed)

# Bi-LSTM theo như RCNN
forward = LSTM(256, return_sequences=True)(l_embedding)
backward = LSTM(256, return_sequences=True, go_backwards=True)(r_embedding)
backward = Lambda(lambda x: tf.reverse(x, axis=[1]))(backward)  # reverse theo chiều time-step

# Kết hợp
together = Concatenate(axis=2)([forward, text_embed, backward])

# Semantic convolution
semantic = Conv1D(256, kernel_size=1, activation="tanh")(together)

# Max pooling theo chiều time-step
sentence_embed = Lambda(lambda x: K.max(x, axis=1))(semantic)

# Dense + output
dense_layer = Dense(256, activation='relu')(sentence_embed)
preds = Dense(1, activation='sigmoid')(dense_layer)

# Final model
model1 = Model(inputs, preds)
model1.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=LR),
    metrics=['acc']
)

In [21]:
token_dict = {}
with codecs.open(vocab_path, 'rb','utf-8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

In [22]:
from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict,cased=True)

In [23]:
from sklearn.model_selection import train_test_split
def load_data(data, sentiments):
    global tokenizer
    indices = []
    segments = []  # Add a list to store segment IDs
    for text in data:
        ids, segs = tokenizer.encode(text, max_len=SEQ_LEN)
        indices.append(ids)
        segments.append(segs)  # Store segment IDs

    # Return both indices and segments
    return [np.array(indices), np.array(segments)], np.array(sentiments)

In [24]:
data_train_,data_valid,label_train_,label_valid = train_test_split(data_train,label_train,test_size=0.15,random_state=48)

In [25]:
X_train,Y_train = load_data(data_train_,label_train_)
X_valid,Y_valid = load_data(data_valid,label_valid)
X_test,Y_test = load_data(data_test,label_test)

In [26]:
checkpoint_path = path + '256_checkpoint_rcnn2_foody.h5'

In [ ]:
if os.path.exists(checkpoint_path):
    print("🔁 Đang load lại model từ checkpoint...")
    model1 = load_model(checkpoint_path)
else:
    print("🚀 Không có checkpoint, train từ đầu.")

🚀 Không có checkpoint, train từ đầu.


In [27]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    path + '256_checkpoint_rcnn2_foody.h5',
    monitor='val_acc',  # Giữ nguyên như bạn muốn
    verbose=1,
    save_best_only=True,
    mode='max'
)
callbacks_list = [checkpoint]


In [ ]:
# model1.load_weights(path+'256_checkpoint_rcnn2_foody.h5')

In [ ]:
# if len(Y_train) != X_train.shape[0]:
#     raise ValueError("Số lượng mẫu trong X_train và Y_train không khớp")
# print(f"Shape của từng phần tử X_train: {[x.shape for x in X_train]}")
# print(f"Shape của Y_train: {Y_train.shape}")

In [28]:
model1.fit(
    X_train, Y_train,
    epochs=1,  # hoặc 1, mỗi lần chạy từng phần nhỏ
    batch_size=16,
    verbose=1,
    validation_data=(X_valid, Y_valid),
    callbacks=callbacks_list
)


KeyboardInterrupt: 